In [1]:
#featsNew - 820 ebs - with 420 that my algorithm missed according to AMC and 400 randomly sampled eb from our original list
#Try to take a random sample and add that to the list of 400 missed ebs to use as training set
#imports
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
#set up
warnings.filterwarnings("ignore", category=RuntimeWarning) #fset = pd.read_csv('sect6data.csv')
data_dir = "/mnt/disks/lcs/"
data.mount_drive(data_dir)
data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"

Disk mounted


In [3]:
#import sector 6 features
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint
datafile = "/home/jupyter/mountpoint/dataproducts/db.v2.h5"
scores6 = pd.read_hdf(datafile, f'S6/scores')
features6 = pd.read_hdf(datafile, f'S6/features')

2022/12/13 22:47:51.759312 Start gcsfuse/0.39.2 (Go version go1.17.5) for app "" using mount point: /home/jupyter/mountpoint
2022/12/13 22:47:51.770915 Opening GCS connection...
2022/12/13 22:47:51.888266 Mounting file system "tess-goddard-lcs"...
2022/12/13 22:47:51.953181 File system has been successfully mounted.


In [4]:
#use list of 400 here
#400 ebs
txt_file = open("TestNewDataset/EBs_noYifan_sector6.txt", "r")
file_content = txt_file.read()
content_list = file_content.split("\n")
txt_file.close()
eb = [eval(i) for i in content_list]

#list of eb tics
eb

[235302840,
 291385340,
 151467181,
 100682876,
 72782194,
 61399839,
 61587373,
 61587383,
 237652393,
 237744078,
 42819010,
 437956145,
 434400399,
 119553244,
 36995455,
 97642729,
 25248826,
 220113180,
 220113185,
 717187734,
 717187735,
 202015589,
 307021664,
 307021665,
 237816806,
 266005231,
 711834166,
 279902112,
 762634460,
 91870711,
 91870715,
 91870721,
 237742115,
 434403321,
 250589534,
 263694657,
 263694668,
 63043258,
 168195898,
 282304507,
 53176760,
 231347373,
 250470383,
 282607438,
 282607443,
 36998354,
 438490744,
 319730823,
 167119003,
 291332565,
 97643944,
 199886262,
 237667069,
 263805896,
 53421164,
 235449761,
 237613397,
 237635393,
 301775577,
 427394717,
 427394721,
 427394732,
 333140513,
 386267565,
 120327872,
 438193125,
 30313695,
 282303787,
 36438546,
 167307525,
 280297598,
 207019236,
 234256682,
 237816792,
 386045551,
 206858968,
 151281308,
 237422089,
 281413326,
 317333116,
 124096035,
 143105037,
 220212798,
 234270718,
 237426004

In [5]:
f6eb = features6.loc[eb]

In [57]:
f6eb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
235302840,0.013395,0.996652,0.000179,0.013440,82.0,0.0,15.0,15.0,0.002874,0.246339,...,0.832604,0.051611,0.092556,0.131819,0.185623,0.315741,0.107240,0.275215,1.160,0.753340
291385340,0.072373,0.983047,0.005238,0.073621,50.0,0.0,24.0,24.0,0.000392,0.176828,...,0.935449,0.002461,0.004356,0.006597,0.010002,0.018724,0.478312,0.034140,1.156,0.263096
151467181,0.037922,0.987686,0.001438,0.038394,143.0,0.0,0.0,0.0,0.011252,0.284501,...,0.620130,0.053468,0.106483,0.182192,0.336230,0.779869,0.148090,0.210749,1.147,0.659157
100682876,0.041641,0.988516,0.001734,0.042125,86.0,0.0,1.0,1.0,0.001529,0.282320,...,0.831590,0.012968,0.024442,0.036019,0.142922,0.588100,0.968704,0.162009,1.146,0.862134
72782194,0.023859,0.992804,0.000569,0.024032,92.0,0.0,15.0,15.0,0.006502,0.150122,...,0.728814,0.053549,0.109268,0.215764,0.327586,0.493540,0.126629,0.145660,1.145,0.144866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277027757,0.109855,0.960602,0.012068,0.114361,197.0,0.0,1.0,1.0,0.047137,0.753232,...,0.532170,0.124688,0.269331,0.429878,0.639504,0.852260,0.786684,0.490785,0.803,0.646020
176455333,0.022627,0.998778,0.000512,0.022655,126.0,0.0,21.0,21.0,0.008788,0.283753,...,0.644068,0.133602,0.250427,0.354559,0.449061,0.608530,0.114384,0.220071,0.678,0.841343
712832712,0.022627,0.998778,0.000512,0.022655,126.0,0.0,21.0,21.0,0.008788,0.283753,...,0.644068,0.133602,0.250427,0.354559,0.449061,0.608530,0.114384,0.220071,0.678,0.841343
220066690,0.022493,0.996690,0.000506,0.022568,29.0,0.0,20.0,20.0,0.003628,0.116200,...,0.961864,0.116308,0.218254,0.333650,0.478533,0.677794,0.163868,0.074705,0.746,0.770852


In [6]:
#list of ebs that we will use for our training data aside from 400
txt_file = open("data/ebs_s6.txt", "r")
file_content = txt_file.read()
content_list = file_content.split("\n")
txt_file.close()
eborig = [eval(i) for i in content_list]

#list of eb tics
eborig

[235231072,
 301185875,
 301185877,
 234169663,
 89016404,
 80836949,
 140336965,
 156501953,
 282503011,
 715766691,
 301185891,
 436619704,
 39354700,
 220140809,
 156241056,
 285000038,
 285000042,
 234065348,
 265899106,
 39838127,
 234878308,
 43144900,
 206794153,
 265715239,
 207004915,
 235548197,
 247681431,
 36288537,
 119461266,
 234169642,
 237590010,
 39667795,
 235552142,
 79635172,
 369673753,
 280212098,
 266193357,
 143215473,
 234926584,
 445955531,
 712487688,
 279900791,
 387690877,
 42991740,
 438545375,
 92009366,
 159086692,
 247556517,
 284199394,
 231161386,
 445955532,
 445955536,
 206840164,
 206840192,
 234158088,
 308387164,
 60242904,
 714514062,
 139476794,
 139476806,
 308169172,
 159057212,
 309908733,
 349480507,
 42885762,
 206840156,
 206850953,
 234146623,
 308169197,
 60242900,
 220149256,
 231959604,
 42991745,
 140209204,
 237647557,
 264594192,
 264594193,
 264594194,
 281157204,
 138904948,
 280304732,
 437918915,
 97568437,
 247681430,
 369681

In [7]:
from random import sample  
f6eb2 = features6.loc[sample(eborig,400)]
f6eb2

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
453082403,0.161839,0.946331,0.026192,0.171017,387.0,0.0,1.0,1.0,0.112244,5.321927,...,0.192670,0.237824,0.446088,0.638038,0.822010,0.951329,0.981165,0.395945,0.655,0.919207
231160641,0.032538,0.991052,0.001059,0.032831,49.0,0.0,25.0,25.0,0.003130,0.222409,...,0.888403,0.044044,0.083480,0.119530,0.179030,0.524788,0.191830,0.095529,1.226,0.566409
234212259,0.027027,0.992378,0.000730,0.027235,67.0,0.0,18.0,18.0,0.000846,0.098969,...,0.910753,0.006714,0.013021,0.021200,0.028641,0.132536,0.153710,0.026351,1.380,0.778469
39354694,0.027643,0.989852,0.000764,0.027927,115.0,0.0,1.0,1.0,0.002581,0.249806,...,0.834746,0.019078,0.035849,0.057860,0.123085,0.575007,0.121415,0.071847,1.472,0.578189
168109107,0.079935,0.970574,0.006390,0.082359,98.0,0.0,0.0,0.0,0.018095,0.533134,...,0.776344,0.063390,0.111011,0.153723,0.210651,0.508503,0.327632,0.106624,1.375,0.766637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711027655,0.065884,0.974136,0.004341,0.067634,111.0,0.0,0.0,0.0,0.013268,0.822848,...,0.723656,0.033278,0.089980,0.197695,0.316540,0.586700,0.266050,0.447693,1.334,0.891620
437960821,0.006593,0.998012,0.000043,0.006606,79.0,0.0,17.0,17.0,0.000695,0.075851,...,0.890591,0.022085,0.041440,0.068021,0.104783,0.265800,0.043335,0.094891,1.403,0.542929
385899979,0.155753,0.946097,0.024259,0.164627,85.0,0.0,14.0,14.0,0.015793,0.688573,...,0.846827,0.028034,0.045629,0.072219,0.139770,0.348887,0.723119,0.055674,1.458,0.828088
42888681,0.014842,0.995473,0.000220,0.014909,105.0,0.0,16.0,16.0,0.003206,0.168431,...,0.803226,0.051873,0.093428,0.140051,0.248075,0.591255,0.072851,0.138260,1.256,0.664727


In [8]:
#list of ebs from villanova (should be exhaustive in tmcl), which we will use to find our noneb dataset
tmclebs = pd.read_csv('data/allsectorstmclebs.csv')

ref = pd.read_csv(data_dir+f"sector6lookup.csv")
tmcl = ["2_min_cadence" in fn for fn in ref.Filename]
reftmcl = ref[tmcl]
z = ~reftmcl.TIC_ID.isin(tmclebs.TIC_ID.values)
f6nonebs = reftmcl[z].sample(frac=0.25)

In [9]:
f6noneb = features6.loc[f6nonebs.TIC_ID.values]

In [122]:
#new dataset (3) = f6eb + f6noneb
#frames = [f6noneb, f6eb, f6eb2]
frames = [f6noneb, f6eb]
feats = pd.concat(frames)
feats.to_csv('TestNewDataset/featsNew400+allNonEB4.csv')

In [10]:
#run this line to read in feats.csv
feats = pd.read_csv('TestNewDataset/featsNew400+allNonEB4.csv', index_col = 0)

In [12]:
train, test = train_test_split(np.arange(len(feats)), train_size=0.8)
labels = []

for i in range(len(f6noneb)):
    labels.append('NonEB')
for i in range(len(f6noneb),len(f6eb)+len(f6noneb)+len(f6eb2)):
    labels.append('EB')
    
X_trainfull = feats.iloc[train]
X_testfull = feats.iloc[test]

y_train = []
y_test = []
for i in train:
    y_train.append(labels[i])
for i in test:
    y_test.append(labels[i])
    
y_train = pd.Series(data = y_train, index = train)
y_test = pd.Series(data = y_test, index = test)

In [13]:
newy_train = []
for i in (y_train.index.values):
    if (y_train[i]=='NonEB'):
        newy_train.append(0)
    else:
        newy_train.append(1)
y_train = pd.Series(data = newy_train, index = train)

newy_test = []
for i in (y_test.index.values):
    if (y_test[i]=='NonEB'):
        newy_test.append(0)
    else:
        newy_test.append(1)
y_test = pd.Series(data = newy_test, index = test)

In [14]:
X_train = X_trainfull
X_test = X_testfull

In [15]:
RF = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 500, max_depth = 15,random_state=31415))
RF.fit(X_train,y_train)
y_predRF = RF.predict(X_test)
print(accuracy_score(y_test,y_predRF)) # Fraction of classifications that were correct
print(precision_score(y_test,y_predRF,pos_label=1))

0.9989939637826962
1.0


In [35]:
X_train

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
38710225,0.032293,0.998921,1.042825e-03,0.032328,1.0,0.0,1.0,1.0,0.000200,0.014853,...,0.550104,0.147437,0.243414,0.356178,0.488078,0.702640,1.000000,0.379346,0.166,0.182632
167388953,0.001632,1.000003,2.664027e-06,0.001632,293.0,0.0,0.0,0.0,0.000879,0.009961,...,0.438731,0.070810,0.151661,0.291118,0.531075,0.806377,0.005143,0.747723,0.075,0.359403
4380024,0.000149,0.999994,2.209073e-08,0.000149,291.0,0.0,0.0,0.0,0.000100,0.007027,...,0.377119,0.153013,0.270545,0.407155,0.564955,0.761827,0.000569,0.794384,0.127,0.110180
238091193,0.032757,0.998922,1.073001e-03,0.032792,1.0,0.0,1.0,1.0,0.000148,0.013153,...,0.339420,0.145760,0.278648,0.422149,0.589237,0.804850,1.000000,0.757729,0.164,0.040936
201369369,0.032180,0.998951,1.035551e-03,0.032214,1.0,0.0,1.0,1.0,0.000176,0.012214,...,0.440501,0.140609,0.271448,0.400542,0.555035,0.771983,0.996498,0.597813,0.162,0.151884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157091493,0.032854,0.999101,1.079396e-03,0.032884,1.0,0.0,1.0,1.0,0.001115,0.024883,...,0.515608,0.152911,0.276155,0.399982,0.571308,0.778918,1.000000,0.829298,0.143,0.270932
59709368,0.033969,0.998848,1.153916e-03,0.034009,1.0,0.0,1.0,1.0,0.000486,0.024086,...,0.385681,0.158923,0.289155,0.432466,0.586218,0.786080,1.000000,0.653892,0.169,0.193878
31115553,0.032990,0.998901,1.088335e-03,0.033026,1.0,0.0,1.0,1.0,0.000307,0.022632,...,0.372549,0.162432,0.275092,0.403884,0.568813,0.762032,1.000000,0.694673,0.165,0.066408
34853521,0.032878,0.998762,1.080994e-03,0.032919,2.0,0.0,1.0,1.0,0.003632,0.268898,...,0.410901,0.149388,0.264643,0.402662,0.583869,0.792669,1.000000,1.155143,0.154,0.217175


____

In [16]:
#sector you want to classify
sector = 6

In [17]:
#change the variable toClassify to data that you want to classify
toClassify = pd.read_hdf(datafile, f'S{sector}/features')
refsect = pd.read_csv(data_dir+f"sector{sector}lookup.csv")

In [18]:
y_pred = RF.predict(toClassify)
toClassify['pred'] = y_pred
rslt_df4 = toClassify[toClassify['pred'] == 1]
#rslt_df.to_csv(f'Sector{sector}/predSector{sector}.csv')

In [71]:
#frac=1
rslt_df

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred
141126559,0.129283,0.966157,0.016714,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.169932,0.299694,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650,1
31303242,0.071903,0.982412,0.005170,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.003302,0.005697,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295,1
427348923,0.002250,0.999533,0.000005,0.002251,64.0,0.0,25.0,25.0,0.000510,0.036659,...,0.073711,0.144543,0.230340,0.339938,0.525208,0.015149,0.170523,0.959,0.231709,1
50897998,0.002341,0.999006,0.000005,0.002343,162.0,0.0,0.0,0.0,0.000667,0.020193,...,0.075659,0.148590,0.289583,0.481719,0.605978,0.010015,0.205155,1.204,0.815128,1
11756637,0.058857,0.992608,0.003464,0.059295,47.0,0.0,21.0,21.0,0.010426,0.466799,...,0.129199,0.221851,0.312899,0.410307,0.552973,0.429933,0.141874,0.784,0.690770,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220420534,0.046102,0.987261,0.002125,0.046696,78.0,0.0,1.0,1.0,0.003164,0.226328,...,0.022791,0.044569,0.069424,0.185201,0.418781,0.999090,0.079798,1.097,0.696240,1
33716637,0.071296,0.971362,0.005083,0.073398,105.0,0.0,1.0,1.0,0.015595,0.396347,...,0.052635,0.117432,0.247192,0.397989,0.628234,0.997895,0.113634,1.258,0.850790,1
251258191,0.025978,0.992660,0.000675,0.026171,71.0,0.0,26.0,26.0,0.000588,0.072821,...,0.005353,0.009578,0.015233,0.022041,0.049144,0.135237,0.020763,1.469,0.755226,1
39352883,0.003137,0.999276,0.000010,0.003140,48.0,0.0,26.0,26.0,0.000643,0.032524,...,0.092521,0.165339,0.261733,0.382909,0.528774,0.019035,0.099740,1.019,0.840247,1


In [109]:
#frac=0.4
rslt_df2

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred
141126559,0.129283,0.966157,0.016714,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.169932,0.299694,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650,1
31303242,0.071903,0.982412,0.005170,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.003302,0.005697,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295,1
427348923,0.002250,0.999533,0.000005,0.002251,64.0,0.0,25.0,25.0,0.000510,0.036659,...,0.073711,0.144543,0.230340,0.339938,0.525208,0.015149,0.170523,0.959,0.231709,1
50897998,0.002341,0.999006,0.000005,0.002343,162.0,0.0,0.0,0.0,0.000667,0.020193,...,0.075659,0.148590,0.289583,0.481719,0.605978,0.010015,0.205155,1.204,0.815128,1
11756637,0.058857,0.992608,0.003464,0.059295,47.0,0.0,21.0,21.0,0.010426,0.466799,...,0.129199,0.221851,0.312899,0.410307,0.552973,0.429933,0.141874,0.784,0.690770,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33716637,0.071296,0.971362,0.005083,0.073398,105.0,0.0,1.0,1.0,0.015595,0.396347,...,0.052635,0.117432,0.247192,0.397989,0.628234,0.997895,0.113634,1.258,0.850790,1
251258191,0.025978,0.992660,0.000675,0.026171,71.0,0.0,26.0,26.0,0.000588,0.072821,...,0.005353,0.009578,0.015233,0.022041,0.049144,0.135237,0.020763,1.469,0.755226,1
39352883,0.003137,0.999276,0.000010,0.003140,48.0,0.0,26.0,26.0,0.000643,0.032524,...,0.092521,0.165339,0.261733,0.382909,0.528774,0.019035,0.099740,1.019,0.840247,1
34397579,0.129846,0.989043,0.016860,0.131284,148.0,5.0,0.0,5.0,0.021386,0.292172,...,0.033845,0.054847,0.086130,0.256070,0.460708,0.597429,0.752603,0.409,0.162228,1


In [110]:
#frac=0.25
rslt_df3

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred
141126559,0.129283,0.966157,0.016714,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.169932,0.299694,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650,1
30631330,0.102360,0.982971,0.010478,0.104133,398.0,0.0,1.0,1.0,0.085288,2.839757,...,0.270387,0.476478,0.678669,0.842133,0.951993,0.981558,0.420469,0.356,0.931920,1
31303242,0.071903,0.982412,0.005170,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.003302,0.005697,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295,1
427348923,0.002250,0.999533,0.000005,0.002251,64.0,0.0,25.0,25.0,0.000510,0.036659,...,0.073711,0.144543,0.230340,0.339938,0.525208,0.015149,0.170523,0.959,0.231709,1
50897998,0.002341,0.999006,0.000005,0.002343,162.0,0.0,0.0,0.0,0.000667,0.020193,...,0.075659,0.148590,0.289583,0.481719,0.605978,0.010015,0.205155,1.204,0.815128,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37489178,0.007128,0.998871,0.000051,0.007136,20.0,0.0,13.0,13.0,0.000445,0.030928,...,0.082436,0.148011,0.244128,0.374360,0.542184,0.074564,0.031632,0.775,0.449933,1
39352883,0.003137,0.999276,0.000010,0.003140,48.0,0.0,26.0,26.0,0.000643,0.032524,...,0.092521,0.165339,0.261733,0.382909,0.528774,0.019035,0.099740,1.019,0.840247,1
442992726,0.114241,0.971269,0.013051,0.117621,417.0,0.0,1.0,1.0,0.089749,1.081350,...,0.266802,0.477522,0.660139,0.787485,0.893447,0.998977,0.358254,0.479,0.866054,1
34397579,0.129846,0.989043,0.016860,0.131284,148.0,5.0,0.0,5.0,0.021386,0.292172,...,0.033845,0.054847,0.086130,0.256070,0.460708,0.597429,0.752603,0.409,0.162228,1


In [130]:
#only 420 ebs
rslt_df4

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat,pred
141126559,0.129283,0.966157,0.016714,0.133812,225.0,0.0,1.0,1.0,0.078338,1.612755,...,0.169932,0.299694,0.382946,0.535229,0.865685,0.966863,0.332072,0.809,0.866650,1
31303242,0.071903,0.982412,0.005170,0.073190,58.0,0.0,22.0,22.0,0.000670,0.214146,...,0.003302,0.005697,0.008720,0.011854,0.017514,0.998374,0.007386,1.206,0.673295,1
427348923,0.002250,0.999533,0.000005,0.002251,64.0,0.0,25.0,25.0,0.000510,0.036659,...,0.073711,0.144543,0.230340,0.339938,0.525208,0.015149,0.170523,0.959,0.231709,1
50897998,0.002341,0.999006,0.000005,0.002343,162.0,0.0,0.0,0.0,0.000667,0.020193,...,0.075659,0.148590,0.289583,0.481719,0.605978,0.010015,0.205155,1.204,0.815128,1
11756637,0.058857,0.992608,0.003464,0.059295,47.0,0.0,21.0,21.0,0.010426,0.466799,...,0.129199,0.221851,0.312899,0.410307,0.552973,0.429933,0.141874,0.784,0.690770,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37489178,0.007128,0.998871,0.000051,0.007136,20.0,0.0,13.0,13.0,0.000445,0.030928,...,0.082436,0.148011,0.244128,0.374360,0.542184,0.074564,0.031632,0.775,0.449933,1
39352883,0.003137,0.999276,0.000010,0.003140,48.0,0.0,26.0,26.0,0.000643,0.032524,...,0.092521,0.165339,0.261733,0.382909,0.528774,0.019035,0.099740,1.019,0.840247,1
442992726,0.114241,0.971269,0.013051,0.117621,417.0,0.0,1.0,1.0,0.089749,1.081350,...,0.266802,0.477522,0.660139,0.787485,0.893447,0.998977,0.358254,0.479,0.866054,1
34397579,0.129846,0.989043,0.016860,0.131284,148.0,5.0,0.0,5.0,0.021386,0.292172,...,0.033845,0.054847,0.086130,0.256070,0.460708,0.597429,0.752603,0.409,0.162228,1


In [22]:
output = open("eblistforDaniel.txt", "w+")
for i in range(len(rslt_df4.index.values)):
        output.write(str(rslt_df4.index.values[i]))
        output.write("\n")
output.close()

In [111]:
sec = 6
ref = pd.read_csv(data_dir+"sector6lookup.csv")

mount: /mnt/disks/lcs: /dev/sdc already mounted on /mnt/disks/lcs.


Disk mounted


In [139]:
import random
tic = rslt_df4.index.values
random.shuffle(tic)

In [140]:
ref_subset = ref[ref['TIC_ID'].isin(tic[:250])]

In [141]:
ref_subset

,Filename,RA,Dec,TIC_ID,Sector,Camera,CCD,Magnitude
10585,tesslcs_sector_6_104/2_min_cadence_targets/tes...,104.942224,15.532873,157751050,6,1,3,14.01960
11650,tesslcs_sector_6_104/2_min_cadence_targets/tes...,83.818610,-5.389694,427394772,6,1,1,4.91180
17240,tesslcs_sector_6_104/2_min_cadence_targets/tes...,96.430160,-55.161774,260504147,6,4,1,8.52900
34122,tesslcs_sector_6_104/tesslcs_tmag_10_11/tesslc...,88.116623,8.883317,80839678,6,1,4,10.74570
75474,tesslcs_sector_6_104/tesslcs_tmag_10_11/tesslc...,84.873520,11.989616,127194202,6,1,4,10.44480
...,...,...,...,...,...,...,...,...
2699483,tesslcs_sector_6_104/tesslcs_tmag_14_15/tesslc...,80.810852,-4.598347,4204002,6,1,1,14.91410
2699995,tesslcs_sector_6_104/tesslcs_tmag_14_15/tesslc...,97.033563,-3.257811,42827926,6,1,2,14.70090
2745771,tesslcs_sector_6_104/tesslcs_tmag_8_9/tesslc_2...,97.073273,10.347553,206883875,6,1,3,8.92944
2759757,tesslcs_sector_6_104/tesslcs_tmag_9_10/tesslc_...,103.890274,7.745508,235409201,6,1,3,9.73955


In [142]:
s6_sub_files = data_dir+ref_subset.Filename.values
lcc = loaders.LightCurveCollection(ref_subset.Sector.values,
                                   ref_subset.Camera.values,
                                   s6_sub_files,
                            )
lcc.useCpus = 4
lcc.load_all_lcs()


In [143]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"TestNewDataset/250ofrslt_df4.pdf") as pdf:
    plt.cla()
    for i in range(250):
        lcc.lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()